# A complete view of Crocoddyl Pinocchio front-end
This notebook presents a complete movement optimized for a humanoid robot in various contact states.
We use the Differential Action Model (DAM) floating in contact, with two 6D contacts between the robot feet and the ground. 
The problem is then optimized with FDDP from a simple initial guess.

## Setup

In [6]:
import gepetuto.magic

NB: as for all the tutorials, a magic command %do_not_load is introduced to hide
    the solutions to some questions. Change it for %load if you want to see (and
    execute) the solution.


For this notebook, with need Crocoddyl 2 with Pinocchio 2 or 3, and Meshcat for display, nothing else.

In [7]:
# %load tp4/generated/humanoid_taichi_import
import example_robot_data as robex
import numpy as np
import pinocchio
import crocoddyl


## Load robot and prepare the problem

The robot is classically loaded from example-robot-data. We use Talos.

In [8]:
# %load tp4/generated/humanoid_taichi_loadrobot
# ### Load robot
robot = robex.load("talos")
robot_model = robot.model
# The robot data will be used at config time to define some values of the OCP
robot_data = robot_model.createData()


The movement will feature the hand reaching a target, both feet first in contact, then one foot leaving contact to reach succesively two targets. The COM must stay fixed during the movement. Let's define the quantities for that.

In [9]:
# %load tp4/generated/humanoid_taichi_hyperparameters
# ### Hyperparameters

# Set integration time
DT = 5e-2
T = 40

# Initialize reference state, target and reference CoM

hand_frameName = "gripper_left_joint"
rightFoot_frameName = "right_sole_link"
leftFoot_frameName = "left_sole_link"

# Main frame Ids
hand_id = robot_model.getFrameId(hand_frameName)
rightFoot_id = robot_model.getFrameId(rightFoot_frameName)
leftFoot_id = robot_model.getFrameId(leftFoot_frameName)

# Init state
q0 = robot_model.referenceConfigurations["half_sitting"]
x0 = np.concatenate([q0, np.zeros(robot_model.nv)])

# Reference quantities
pinocchio.framesForwardKinematics(robot_model, robot_data, q0)
comRef = (robot_data.oMf[rightFoot_id].translation + robot_data.oMf[leftFoot_id].translation) / 2
comRef[2] = pinocchio.centerOfMass(robot_model, robot_data, q0)[2].item()

in_world_M_foot_target_1 = pinocchio.SE3(np.eye(3), np.array([0.0, 0.4, 0.0]))
in_world_M_foot_target_2 =  pinocchio.SE3(np.eye(3), np.array([0.3, 0.15, 0.35]))
in_world_M_hand_target = pinocchio.SE3(np.eye(3), np.array([0.4, 0, 1.2]))


## Action models

We define the smaller models first (state, actuation, contacts, costs) and finish with the DAM and action models.

### State and action models 

The state model is classical for legged robots: the state "multibody" defines $x$ to be $x=(q,v_q)$.
The actuation model is also classical: the joint torques are 0 for the free-flyer part, and $u$ for the rest, $\tau_q = (0_6,u)$

In [10]:
# %load tp4/generated/humanoid_taichi_state
state = crocoddyl.StateMultibody(robot_model)
actuation = crocoddyl.ActuationModelFloatingBase(state)


### Contact models

We define one contact model per foot (right, and left). These are then used to define the full configuration mode for each timestep. In this example, the first timesteps are in double contact (right and left together); the final timesteps are in single contact (right only, left foot in the air).

In [11]:
# %load tp4/generated/humanoid_taichi_contacts
# ### Contact model
# Create two contact models used along the motion
supportContactModelLeft = crocoddyl.ContactModel6D(
    state,
    leftFoot_id,
    pinocchio.SE3.Identity(),
    pinocchio.LOCAL,
    actuation.nu,
    np.array([0, 40]),
)
supportContactModelRight = crocoddyl.ContactModel6D(
    state,
    rightFoot_id,
    pinocchio.SE3.Identity(),
    pinocchio.LOCAL,
    actuation.nu,
    np.array([0, 40]),
)

contactModel1Foot = crocoddyl.ContactModelMultiple(state, actuation.nu)
contactModel1Foot.addContact(rightFoot_frameName + "_contact", supportContactModelRight)

contactModel2Feet = crocoddyl.ContactModelMultiple(state, actuation.nu)
contactModel2Feet.addContact(leftFoot_frameName + "_contact", supportContactModelLeft)
contactModel2Feet.addContact(rightFoot_frameName + "_contact", supportContactModelRight)


### Cost models

We define costs for the state and control regularization, hand and left foot targets, COM no-motion, and to limit the range of motion (as a soft penalty).

In [12]:
# %load tp4/generated/humanoid_taichi_costs
# ### Cost model
# Cost for joint limits
maxfloat = 1e25
xlb = np.concatenate(
    [
        -maxfloat * np.ones(6),  # dimension of the SE(3) manifold
        robot_model.lowerPositionLimit[7:],
        -maxfloat * np.ones(state.nv),
    ]
)
xub = np.concatenate(
    [
        maxfloat * np.ones(6),  # dimension of the SE(3) manifold
        robot_model.upperPositionLimit[7:],
        maxfloat * np.ones(state.nv),
    ]
)
bounds = crocoddyl.ActivationBounds(xlb, xub, 1.0)
xLimitResidual = crocoddyl.ResidualModelState(state, x0, actuation.nu)
xLimitActivation = crocoddyl.ActivationModelQuadraticBarrier(bounds)
limitCost = crocoddyl.CostModelResidual(state, xLimitActivation, xLimitResidual)

# Cost for state and control
xResidual = crocoddyl.ResidualModelState(state, x0, actuation.nu)
xActivation = crocoddyl.ActivationModelWeightedQuad(
    np.array([0] * 3 + [10.0] * 3 + [0.01] * (state.nv - 6) + [10] * state.nv) ** 2
)
uResidual = crocoddyl.ResidualModelControl(state, actuation.nu)
xTActivation = crocoddyl.ActivationModelWeightedQuad(
    np.array([0] * 3 + [10.0] * 3 + [0.01] * (state.nv - 6) + [100] * state.nv) ** 2
)
xRegCost = crocoddyl.CostModelResidual(state, xActivation, xResidual)
uRegCost = crocoddyl.CostModelResidual(state, uResidual)
xRegTermCost = crocoddyl.CostModelResidual(state, xTActivation, xResidual)

# Cost for target reaching: hand and foot
handTrackingResidual = crocoddyl.ResidualModelFramePlacement(
    state, hand_id, in_world_M_hand_target, actuation.nu
)
handTrackingActivation = crocoddyl.ActivationModelWeightedQuad(
    np.array([1] * 3 + [0.0001] * 3) ** 2
)
handTrackingCost = crocoddyl.CostModelResidual(
    state, handTrackingActivation, handTrackingResidual
)

# For the flying foot, we define two targets to successively reach
footTrackingResidual1 = crocoddyl.ResidualModelFramePlacement(
    state, leftFoot_id, in_world_M_foot_target_1, actuation.nu
)
footTrackingActivation = crocoddyl.ActivationModelWeightedQuad(
    np.array([1, 1, 0.1] + [1.0] * 3) ** 2
)
footTrackingCost1 = crocoddyl.CostModelResidual(
    state, footTrackingActivation, footTrackingResidual1
)
footTrackingResidual2 = crocoddyl.ResidualModelFramePlacement(
    state,
    leftFoot_id,
    in_world_M_foot_target_2,
    actuation.nu,
)
footTrackingCost2 = crocoddyl.CostModelResidual(
    state, footTrackingActivation, footTrackingResidual2
)

# Cost for CoM reference
comResidual = crocoddyl.ResidualModelCoMPosition(state, comRef, actuation.nu)
comTrack = crocoddyl.CostModelResidual(state, comResidual)


### Action models

We define an action model for each phase of the motion. There are 3 phases, plus a terminal model. For each, we first define a DAM (from costs and contacts), then the integral action model.

In [13]:
# %load tp4/generated/humanoid_taichi_actions
# Create cost model per each action model. We divide the motion in 3 phases plus its
# terminal model.

# Phase 1: two feet in contact, hand reach the target
runningCostModel1 = crocoddyl.CostModelSum(state, actuation.nu)
runningCostModel1.addCost("gripperPose", handTrackingCost, 1e2)
runningCostModel1.addCost("stateReg", xRegCost, 1e-3)
runningCostModel1.addCost("ctrlReg", uRegCost, 1e-4)
dmodelRunning1 = crocoddyl.DifferentialActionModelContactFwdDynamics(
    state, actuation, contactModel2Feet, runningCostModel1
)
runningModel1 = crocoddyl.IntegratedActionModelEuler(dmodelRunning1, DT)

# Phase 2: only right foot in contact, hand stays on target, left foot to target 1
runningCostModel2 = crocoddyl.CostModelSum(state, actuation.nu)
runningCostModel2.addCost("gripperPose", handTrackingCost, 1e2)
runningCostModel2.addCost("footPose", footTrackingCost1, 1e1)
runningCostModel2.addCost("stateReg", xRegCost, 1e-3)
runningCostModel2.addCost("ctrlReg", uRegCost, 1e-4)
dmodelRunning2 = crocoddyl.DifferentialActionModelContactFwdDynamics(
    state, actuation, contactModel1Foot, runningCostModel2
)
runningModel2 = crocoddyl.IntegratedActionModelEuler(dmodelRunning2, DT)

# Phase 3: only right foot in contact, hand stays on target, left foot to target 2
runningCostModel3 = crocoddyl.CostModelSum(state, actuation.nu)
runningCostModel3.addCost("gripperPose", handTrackingCost, 1e2)
runningCostModel3.addCost("footPose", footTrackingCost2, 1e1)
runningCostModel3.addCost("stateReg", xRegCost, 1e-3)
runningCostModel3.addCost("ctrlReg", uRegCost, 1e-4)
dmodelRunning3 = crocoddyl.DifferentialActionModelContactFwdDynamics(
    state, actuation, contactModel1Foot, runningCostModel3
)
runningModel3 = crocoddyl.IntegratedActionModelEuler(dmodelRunning3, DT)

# Terminal cost: nothing specific
terminalCostModel = crocoddyl.CostModelSum(state, actuation.nu)
terminalCostModel.addCost("gripperPose", handTrackingCost, 1e2)
terminalCostModel.addCost("stateReg", xRegTermCost, 1e-3)
dmodelTerminal = crocoddyl.DifferentialActionModelContactFwdDynamics(
    state, actuation, contactModel1Foot, terminalCostModel
)
terminalModel = crocoddyl.IntegratedActionModelEuler(dmodelTerminal, 0)


## Write problem and solve

### Write the OCP problem and create the solve.

The OCP is simply the collection of action models.

In [14]:
# %load tp4/generated/humanoid_taichi_problem_and_solver
problem = crocoddyl.ShootingProblem(
    x0, [runningModel1] * T + [runningModel2] * T + [runningModel3] * T, terminalModel
)

# Creating the DDP solver for this OC problem, defining a logger
solver = crocoddyl.SolverFDDP(problem)
solver.th_stop = 1e-7
solver.setCallbacks(
    [
        crocoddyl.CallbackVerbose(),
        crocoddyl.CallbackLogger(),
    ]
)


### and solve ...

We solve from a nontrivial (but simple) initial trajectory. The states are chosen constant equal to the initial state. The control are compensating the gravity.

In [15]:
# %load tp4/generated/humanoid_taichi_solve
# ### Warm start from quasistatic solutions
xs = [x0] * (solver.problem.T + 1)
us = solver.problem.quasiStatic([x0] * solver.problem.T)
solver.solve(xs, us, 500, False, 1e-9)


iter    cost       merit      stop       grad       preg       dreg      step   ||ffeas||  ||gfeas||  ||hfeas||   dV-exp       dV      dPhi-exp     dPhi
   0  1.685e+02  0.000e+00  1.757e+02  3.514e+02  1.000e-09  1.000e-09  0.0312  9.095e-14  0.000e+00  0.000e+00  1.081e+01  1.054e+01  0.000e+00  0.000e+00
   1  1.591e+02  0.000e+00  1.652e+02  3.303e+02  1.000e-09  1.000e-09  0.0312  8.811e-14  0.000e+00  0.000e+00  1.016e+01  9.396e+00  0.000e+00  0.000e+00
   2  1.506e+02  0.000e+00  1.558e+02  3.116e+02  1.000e-09  1.000e-09  0.0312  8.535e-14  0.000e+00  0.000e+00  9.584e+00  8.466e+00  0.000e+00  0.000e+00
   3  1.424e+02  0.000e+00  1.473e+02  2.947e+02  1.000e-09  1.000e-09  0.0312  8.269e-14  0.000e+00  0.000e+00  9.064e+00  8.211e+00  0.000e+00  0.000e+00
   4  1.341e+02  0.000e+00  1.391e+02  2.782e+02  1.000e-09  1.000e-09  0.0312  8.010e-14  0.000e+00  0.000e+00  8.559e+00  8.251e+00  0.000e+00  0.000e+00
   5  1.261e+02  0.000e+00  1.309e+02  2.618e+02  1.000e-09  1.000e

True

iter    cost       merit      stop       grad       preg       dreg      step   ||ffeas||  ||gfeas||  ||hfeas||   dV-exp       dV      dPhi-exp     dPhi
  70  3.119e+00  0.000e+00  7.956e-08  1.591e-07  1.000e-09  1.000e-09  0.5000  0.000e+00  0.000e+00  0.000e+00  5.967e-08  3.500e-08  0.000e+00  0.000e+00


### Display to finish

In [17]:
# %load tp4/generated/humanoid_taichi_init_display
# ### DISPLAY
# Initialize viewer
from schaeffler2025.meshcat_viewer_wrapper import MeshcatVisualizer
viz = MeshcatVisualizer(robot)
viz.display(robot.q0)


*** You asked to start meshcat "classically" in tcp://127.0.0.1:6000
*** Did you start meshcat manually (meshcat-server)
Wrapper tries to connect to server <tcp://127.0.0.1:6000>
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [18]:
viz.viewer.jupyter_cell()

In [19]:
# %load tp4/generated/humanoid_taichi_display
viz.play([x[:robot.model.nq] for x in solver.xs],DT)
